# Adjusting [whpview](http://www.roy.hi-ho.ne.jp/ssai/mito_gis/whpview/index.html) data format to [GWO](http://www.roy.hi-ho.ne.jp/ssai/mito_gis/) data format
**Author: Jun Sasaki  Coded on February 13, 2022  Updated on December 17, 2023.**<br>
- Note: pvlib required:
```bash
conda install pvlib
```

## 課題
雲量に`0+`があり，要対応．また，雲量は3時間データであるため，データの無いところはNaNにしておく必要がある．

降水量の `--` とデータ無しは欠損値とし，欠損値はすべて0とする．Remarkが存在しないため，欠損値を0とする以外の方法はない．
- 降雪，積雪：削除
- 降水量： `--` => `0`
- 日照時間： `""` => `0`
- 全天日射： `""` => `0`
- 降雪(cm)，積雪(cm)，--，視程：削除
- `["現地気圧(hPa)","海面気圧(hPa)","降水量(mm)","気温(℃)","露点温度(℃)","蒸気圧(hPa)","風速(m/s)","日照時間(h)","全天日射量(MJ/㎡)"]`

In [1]:
from metdata import whp
#import sys
#import os
#import glob
#import numpy as np
#import math
#import pandas as pd
#import datetime
#import subprocess
#from pandas.tseries.offsets import Hour
#from dateutil.parser import parse

降水量の `--` とデータ無しは欠損値とし，欠損値はすべて0とする．Remarkが存在しないため，欠損値を0とする以外の方法はない．
- 降雪，積雪：削除
- 降水量： `--` => `0`
- 日照時間： `""` => `0`
- 全天日射： `""` => `0`
- 降雪(cm)，積雪(cm)，--，視程：削除
- `["現地気圧(hPa)","海面気圧(hPa)","降水量(mm)","気温(℃)","露点温度(℃)","蒸気圧(hPa)","風速(m/s)","日照時間(h)","全天日射量(MJ/㎡)"]`

In [2]:
#fi_path = "/mnt/d/dat/met/JMA_DataBase/whpView/wwwRoot/data/2020/Tokyo_2020.csv"
tokyo_2021 = whp.Whp()
tokyo_2021.df

2021
Tokyo
/mnt/d/dat/met/JMA_DataBase/whpView/wwwRoot/data/2021/Tokyo_2021.csv


,観測所ID,観測所名,ID1,年,月,日,時,現地気圧(0.1hPa),ID2,海面気圧(0.1hPa),...,現在天気,ID10,露天温度(0.1degC),ID11,日照時間(0.1時間),ID12,全天日射量(0.01MJ/m2/h),ID13,降水量(0.1mm/h),ID14
0,662,東京,662,2021,1,1,1,10084,8,10114,...,NaN,0,-98.0,8,NaN,8,NaN,8,0000000000,8
1,662,東京,662,2021,1,1,2,10084,8,10114,...,NaN,0,-99.0,8,NaN,8,NaN,8,0000000000,8
2,662,東京,662,2021,1,1,3,10080,8,10111,...,NaN,0,-92.0,8,NaN,8,NaN,8,0000000000,8
3,662,東京,662,2021,1,1,4,10078,8,10108,...,NaN,0,-88.0,8,NaN,8,NaN,8,0000000000,8
4,662,東京,662,2021,1,1,5,10078,8,10108,...,NaN,0,-84.0,8,NaN,8,NaN,8,0000000000,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,662,東京,662,2021,12,31,20,10116,8,10146,...,NaN,0,-82.0,8,NaN,8,NaN,8,0000000000,8
8756,662,東京,662,2021,12,31,21,10119,8,10149,...,NaN,0,-95.0,8,NaN,8,NaN,8,0000000000,8
8757,662,東京,662,2021,12,31,22,10123,8,10154,...,NaN,0,-93.0,8,NaN,8,NaN,8,0000000000,8
8758,662,東京,662,2021,12,31,23,10121,8,10152,...,NaN,0,-105.0,8,NaN,8,NaN,8,0000000000,8


In [3]:
tokyo_2021.to_csv()

Saving to /mnt/d/dat/met/JMA_DataBase/GWO/Hourly/Tokyo/Tokyo2021.csv


In [4]:
tokyo_2021.df_org

,名称,観測所ID,日付,時,現地気圧(hPa),海面気圧(hPa),降水量(mm),気温(℃),露点温度(℃),蒸気圧(hPa),湿度(%),風速(m/s),風向,日照時間(h),全天日射量(MJ/㎡),雲量
0,東京,47662,2021/01/01,1,1008.4,1011.4,0,0.8,-9.8,2.9,45.0,1.6,北西,NaN,NaN,NaN
1,東京,47662,2021/01/01,2,1008.4,1011.4,0,0.4,-9.9,2.9,46.0,1.5,北西,NaN,NaN,NaN
2,東京,47662,2021/01/01,3,1008.0,1011.1,0,-0.6,-9.2,3.0,52.0,1.1,西北西,NaN,NaN,0
3,東京,47662,2021/01/01,4,1007.8,1010.8,0,0.1,-8.8,3.1,51.0,1.8,北西,NaN,NaN,NaN
4,東京,47662,2021/01/01,5,1007.8,1010.8,0,0.0,-8.4,3.2,53.0,0.6,北西,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,東京,47662,2021/12/31,20,1011.6,1014.6,0,1.0,-8.2,3.3,50.0,1.2,北北西,NaN,NaN,NaN
8756,東京,47662,2021/12/31,21,1011.9,1014.9,0,1.4,-9.5,3.0,44.0,2.9,北西,NaN,NaN,0
8757,東京,47662,2021/12/31,22,1012.3,1015.4,0,0.7,-9.3,3.0,47.0,2.5,北北西,NaN,NaN,NaN
8758,東京,47662,2021/12/31,23,1012.1,1015.2,0,0.6,-10.5,2.7,43.0,2.5,北北西,NaN,NaN,NaN
